In [ ]:
!pip install datasets
!pip install transformers==4.30
!pip install transformers[torch]
import pandas as pd
from sklearn.model_selection import train_test_split

import random
import math

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from transformers import GPT2Tokenizer, GPT2Model


from datasets import load_dataset
from datasets import ClassLabel
from datasets import Dataset


from IPython.display import display, HTML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_train = '/content/drive/MyDrive/train_final.csv'
path_to_validation = '/content/drive/MyDrive/test_final.csv'
datasets = load_dataset("csv", data_files={"train": path_to_train, "validation": path_to_validation})


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

In [ ]:
train_tokens = []

for lyric in datasets['train']:
    try:
        tokens = tokenizer(text=lyric['Lyrics'])
        train_tokens.append(tokens)
    except:
        continue

train_tokens = Dataset.from_pandas(pd.DataFrame(train_tokens))


test_tokens = []

for lyric in datasets['validation']:
    try:
        tokens = tokenizer(lyric["Lyrics"])
        test_tokens.append(tokens)
    except:
        continue
test_tokens = Dataset.from_pandas(pd.DataFrame(test_tokens))

Token indices sequence length is longer than the specified maximum sequence length for this model (1050 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
block_size = 256

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset_train = train_tokens.map(group_texts,batched=True,batch_size=8)
lm_dataset_test = test_tokens.map(group_texts,batched=True,batch_size=8)

Map:   0%|          | 0/5549 [00:00<?, ? examples/s]

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

In [ ]:
lm_dataset_train

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 18967
})

In [ ]:

model_name='distilgpt2'

model_name_new = model_name.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name_new}-rap-bot",
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    num_train_epochs=20,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps = 100.0,
)


In [ ]:
model_name="distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import torch

# make sure cuda gpu is available or training (makes it way faster)
print(torch.__version__)
print(torch.cuda.is_available())

print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

2.2.1+cu121
True
1
0
Tesla T4


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train,
    eval_dataset=lm_dataset_test
)

In [1]:
trainer.train()

NameError: name 'trainer' is not defined